# Streaming

<img src="./assets/LC_streaming.png" width="400">

Streaming reduces the latency between generating data and the user receiving it.
There are two types frequently used with Agents:

## Setup

Load and/or check for needed environmental variables

In [1]:
from dotenv import load_dotenv
from env_utils import doublecheck_env

# Load environment variables from .env
load_dotenv()

# Check and print results
doublecheck_env("example.env")

DASHSCOPE_API_KEY=****7258
LANGSMITH_API_KEY=****4e15
LANGSMITH_TRACING=true
LANGSMITH_PROJECT=****ials


In [3]:
from langchain.agents import create_agent

In [5]:
from langchain_community.chat_models import ChatTongyi

# Initialize Qwen model via DashScope (Alibaba Cloud)
model = ChatTongyi(model="qwen-turbo")

agent = create_agent(
    model=model,
    system_prompt="You are a full-stack comedian",
)

## No Steaming (invoke)

In [6]:
result = agent.invoke({"messages": [{"role": "user", "content": "Tell me a joke"}]})
print(result["messages"][1].content)

Ah, a joke? Let me think... I've got one that's so good, it'll make your brain do a backflip and land in a pool of confusion.

Why don't skeletons ever get cold?

Because they don't have the *guts* to go out in the cold!

*(pauses for effect)*

Wait, no... that's not it. That's the one I told at the family reunion and my aunt still hasn't forgiven me.

Okay, let me try again. Here's a real one:

I told my computer I needed a break, and it said, "You're 100% correct. Restarting now."

*(leans in conspiratorially)*

But seriously, if you ever feel like the world is against you, just remember: you're not alone. The moon is always there, judging you from afar.


## values
You have seen this streaming mode in our examples so far. 

In [7]:
# Stream = values
for step in agent.stream(
    {"messages": [{"role": "user", "content": "Tell me a Dad joke"}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Tell me a Dad joke
================================== Ai Message ==================================

Why don't skeletons fight each other?  
They don't have the *guts*! 😄

(And also, they’re already *dead* in the ring.)


## messages
Messages stream data token by token - the lowest latency possible. This is perfect for interactive applications like chatbots.

In [8]:
for token, metadata in agent.stream(
    {"messages": [{"role": "user", "content": "Write me a family friendly poem."}]},
    stream_mode="messages",
):
    print(f"{token.content}", end="")

Sure! Here's a family-friendly poem that's light, cheerful, and perfect for all ages:

---

**"The Day We All Went Barking"**

One sunny morning, bright and clear,  
We woke up with a curious cheer.  
Mom said, "Let’s go for a walk!"  
Dad said, "I think I’ll *woof* a bit."  

We all got dressed in silly clothes,  
With hats on backwards, and socks on toes.  
Sister wore a hat made of spaghetti,  
And brother had a banana on his head.  

We went outside to chase the breeze,  
And suddenly—*what was that noise?*  
A bark! A yip! A howl! A roar!  
We all started to *bark* and *roar*!  

We barked at clouds, we barked at trees,  
We barked at squirrels and even bees.  
We barked so loud, the moon smiled wide,  
And stars all giggled in the sky.  

We barked until our cheeks were sore,  
Then stopped and looked at each other.  
"We’re not dogs," we said with a grin,  
"But *barking* is the best thing ever!"  

So if you feel a little silly,  
Or need a laugh, or just a thrill,  
Just take 

## Tools can stream too!
Streaming generally means delivering information to the user before the final result is ready. There are many cases where this is useful. A `get_stream_writer` writer allows you to easily stream `custom` data from sources you create.

In [9]:
from langchain.agents import create_agent
from langgraph.config import get_stream_writer


def get_weather(city: str) -> str:
    """Get weather for a given city."""
    writer = get_stream_writer()
    # stream any arbitrary data
    writer(f"Looking up data for city: {city}")
    writer(f"Acquired data for city: {city}")
    return f"It's always sunny in {city}!"


agent = create_agent(
    model=model,
    tools=[get_weather],
)

for chunk in agent.stream(
    {"messages": [{"role": "user", "content": "What is the weather in SF?"}]},
    stream_mode=["values", "custom"],
):
    print(chunk)

('values', {'messages': [HumanMessage(content='What is the weather in SF?', additional_kwargs={}, response_metadata={}, id='adb6d41e-77c8-40be-8736-894cd9eb4e59')]})
('values', {'messages': [HumanMessage(content='What is the weather in SF?', additional_kwargs={}, response_metadata={}, id='adb6d41e-77c8-40be-8736-894cd9eb4e59'), AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"city": "SF"}', 'name': 'get_weather'}, 'id': 'call_297f45d6a3bf436e89a3c6', 'index': 0, 'type': 'function'}]}, response_metadata={'model_name': 'qwen-turbo', 'finish_reason': 'tool_calls', 'request_id': '0566db64-1181-4e75-b844-566455432b10', 'token_usage': {'input_tokens': 157, 'output_tokens': 19, 'prompt_tokens_details': {'cached_tokens': 0}, 'total_tokens': 176}}, id='lc_run--30c50d84-6d88-4660-8db2-5b0610482674-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'SF'}, 'id': 'call_297f45d6a3bf436e89a3c6', 'type': 'tool_call'}])]})
('custom', 'Looking up data for city:

In [10]:
for chunk in agent.stream(
    {"messages": [{"role": "user", "content": "What is the weather in SF?"}]},
    stream_mode=["custom"],
):
    print(chunk)

('custom', 'Looking up data for city: SF')
('custom', 'Acquired data for city: SF')


## Try different modes on your own!
Modify the stream mode and the select to produce different results.

In [ ]:
for chunk in agent.stream(
    {"messages": [{"role": "user", "content": "What is the weather in SF?"}]},
    stream_mode=["values", "custom"],
):
    if chunk[0] == "custom":
        print(chunk[1])